In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from lib import utils

In [6]:
args = pickle.load(open('arguments.pickle', 'rb'))

# position each protein

In [3]:
# !pip install bcbio-gff 

In [4]:
# import pprint
# from BCBio.GFF import GFFExaminer
# from BCBio import GFF

In [5]:
# gfffile = "../COVID_data/GCF_009858895.2_ASM985889v3_genomic.gff"

# position_gene = []

# featureid_locations={}
# limits=dict(gff_type=["gene","mRNA","CDS"])
# with open(gfffile) as in_handle:
#     for rec in GFF.parse(in_handle, limit_info=limits):
#         for feature in rec.features:
#             print(feature.qualifiers['Name'],feature.location.start,feature.location.end)
#             position_gene += [[str(feature.qualifiers['Name'][0]),feature.location.start,feature.location.end]]
# #             print(feature.qualifiers['Name'])
# #             print(feature.qualifiers)


In [9]:
gene_data = pd.read_csv(args.reference_gene_loci, sep='\t')

In [10]:
gene_df = pd.DataFrame(gene_data)
gene_df.head()


,gene,start,end,#AA
0,NSP1,266,805,180
1,NSP2,806,2719,638
2,NSP3,2720,8554,1944
3,NSP4,8555,10054,499
4,NSP5,10055,10972,605


In [11]:
gene_df.loc[:,'start']-=1
#start ที่่ 0

In [12]:
gene_df.head()

,gene,start,end,#AA
0,NSP1,265,805,180
1,NSP2,805,2719,638
2,NSP3,2719,8554,1944
3,NSP4,8554,10054,499
4,NSP5,10054,10972,605


In [13]:
gene_df.to_csv(os.path.join(args.preprocess_dir, 'position_gene_df.csv'), index=False) 

# check amino change

In [14]:
seq_ref_list = pickle.load(open(os.path.join(args.preprocess_dir, "seq_ref_list.pickle"), "rb"))
align_array_df = pickle.load(open(os.path.join(args.preprocess_dir, 'align_array_df.pickle'), 'rb'))

In [16]:
snp_max_df = pd.read_csv(os.path.join(args.preprocess_dir, 'snps_max_df.csv'))

# snp_max_df = snp_max_df[snp_max_df['sbjct']!='-']
snp_max_df = snp_max_df[snp_max_df['sbjct']!='N']
snp_max_df = snp_max_df.sort_values('id')

snp_max_df = snp_max_df.reset_index()

In [17]:
change_df = snp_max_df.copy()
change_df = change_df[['id','position','query','sbjct']]
change_df['position'] -= 1 #ลบเพราะจะเทียบกับ ref ซึ่่งเริ่มที่ 0
change_df['check']='None'

In [18]:
change_protein_df = utils.change_protein_table(change_df,gene_df,align_array_df,seq_ref_list)
change_protein_df = change_protein_df[change_protein_df['check']==False].drop_duplicates(keep='first')

change_protein_df.head()

,id,pos_neucleotide,query_protein,position_protein,sbjct_protein,change_protein,gene,check
1,407987,27147,D,209,H,D209H,M,False
2,407988,11083,L,37,F,L37F,NSP6,False
4,410535,28144,L,84,S,L84S,ORF8,False
5,410535,28878,S,202,N,S202N,N,False
7,410536,26144,G,251,V,G251V,ORF3a,False


In [19]:
#change position & name gene NSP12
change_protein_df.loc[change_protein_df['gene']=='NSP12_2','position_protein']+=9
change_protein_df.loc[change_protein_df['gene']=='NSP12_2','gene']= 'NSP12'
change_protein_df.loc[change_protein_df['gene']=='NSP12_1','gene']= 'NSP12'

In [20]:
change_protein_df['change_protein'] = [str(q)+str(p)+str(s) for q,p,s in zip(change_protein_df['query_protein'],change_protein_df['position_protein'],change_protein_df['sbjct_protein'])]

In [21]:
change_protein_df.to_csv(os.path.join(args.preprocess_dir, 'summary_change_protein_df.csv'), index=False) 

In [23]:
change_protein_df=pd.read_csv(os.path.join(args.preprocess_dir, 'summary_change_protein_df.csv')) 

In [24]:
change_protein_df

,id,pos_neucleotide,query_protein,position_protein,sbjct_protein,change_protein,gene,check
0,407987,27147,D,209,H,D209H,M,False
1,407988,11083,L,37,F,L37F,NSP6,False
2,410535,28144,L,84,S,L84S,ORF8,False
3,410535,28878,S,202,N,S202N,N,False
4,410536,26144,G,251,V,G251V,ORF3a,False
...,...,...,...,...,...,...,...,...
77886,2887795,19220,A,394,V,A394V,NSP14,False
77887,2887795,15451,G,671,S,G671S,NSP12,False
77888,2887795,14408,P,323,L,P323L,NSP12,False
77889,2887795,28253,F,120,-,F120-,ORF8,False
